<h1>Getting Started</h1>

<h2>One-Time Setup</h2>

To get through this, you'll need the following:
<ul>
    <li> NumPy
    <li> TensorFlow
    <li> MatplotLib
</ul>

We highly recommend using `pip` for easy installation (even if you're on Windows like me). In Terminal:<br>
`pip install numpy`<br>
`pip install tensorflow`<br>
`pip -mpip install matplotlib`<br>

For those of you who already have versions of these packages, you can update with<br>
`pip install --upgrade <package>`

If you encounter issues with dependencies that you've already installed (e.g. you're trying to install tensorFlow but it tells you there's some issue with one of its dependencies), try<br>
`pip install --ignore-installed <package>`

<h2>Every-Time Setup</h2>

In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

This tutorial has a few exercises meant to familiarize you with the tools we'll be using and is primarily focused on implementation. It will assume familarity with linear algebra basics (convolution, BLAS) and Python syntax. For this research project, much of the infrastructure will be set up for you already, but what's covered here will make life much faster later on.

That said, let's begin!

<h2>Exercise: Implementing a Basic Neural Network for Character Recognition</h2>

Adapted from TensorFlow's <a href="https://www.tensorflow.org/tutorials/layers">layers tutorial</a>.

<a href="http://yann.lecun.com/exdb/lenet/">LeNet-5</a> is a convolutional neural network (CNN) designed for handwritten and machine-printed character recognition. Its structure looks like so:

<center>
    <img src="./figs/arch-lenet5.png">
</center>

Each of these stages is a layer of the neural network, and they can take on a few different flavors. Our friends across the Bay have provided a <a href="http://cs231n.github.io/convolutional-networks/#layers">good explanation</a> of the different types of layers used in CNNs, and we strongly suggest at least skimming it to get a general idea of the computation involved with each layer. We'll be using TensorFlow to implement Lenet-5 and train it to identify handwritten characters using the <a href="http://yann.lecun.com/exdb/mnist/">MNIST dataset</a>.<p>

<center>
    <img src="./figs/onions.png", style='width:500px'>
</center>


In [2]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.3006723, step = 1
INFO:tensorflow:probabilities = [[0.09671545 0.1291725  0.06805394 0.11180378 0.09437028 0.08973049
  0.09460642 0.10617087 0.10525422 0.1041221 ]
 [0.09512156 0.10985074 0.08080811 0.11738139 0.11449763 0.08852433
  0.11029475 0.08057559 0.09463713 0.10830867]
 [0.09697781 0.11253522 0.0884854  0.10975073 

INFO:tensorflow:probabilities = [[0.09025025 0.11989091 0.07283956 0.10915705 0.09798737 0.08590054
  0.10254981 0.10183106 0.09465772 0.12493561]
 [0.08330182 0.11218413 0.08754308 0.10331307 0.11003508 0.09630531
  0.09647696 0.08960991 0.10193151 0.11929914]
 [0.09113932 0.11276294 0.09478658 0.09940495 0.09579581 0.09813819
  0.10110417 0.08950739 0.0946465  0.1227142 ]
 [0.09067339 0.11113728 0.08496934 0.10663267 0.10451964 0.08193621
  0.11669303 0.09401035 0.09574531 0.11368275]
 [0.09669709 0.10104255 0.09202321 0.10895533 0.11109794 0.09520172
  0.10759093 0.08411399 0.09044804 0.11282921]
 [0.10162261 0.11495543 0.08677639 0.09607747 0.10665398 0.08996642
  0.09756827 0.10692566 0.09170551 0.10774826]
 [0.10117386 0.10354181 0.09855164 0.10818633 0.09506747 0.09036333
  0.10630437 0.10227399 0.09614873 0.09838848]
 [0.09864547 0.10020051 0.08674697 0.10500204 0.1063441  0.0940387
  0.1135582  0.09234105 0.10077556 0.10234747]
 [0.08940189 0.1187181  0.10674356 0.09101732 0.0

INFO:tensorflow:global_step/sec: 2.59922
INFO:tensorflow:loss = 2.3056138, step = 101 (38.474 sec)
INFO:tensorflow:probabilities = [[0.09924741 0.11279572 0.10069145 0.09551296 0.09162287 0.092097
  0.10309637 0.10020671 0.08957075 0.11515881]
 [0.09180816 0.10332928 0.08097081 0.10828113 0.11060242 0.07981215
  0.10959802 0.10851315 0.09089818 0.11618671]
 [0.09424414 0.09553524 0.09453057 0.10504425 0.09172571 0.10020973
  0.1018118  0.10839159 0.10296784 0.10553912]
 [0.09417614 0.11538583 0.09670315 0.10966855 0.09977723 0.08759838
  0.10601117 0.08347197 0.10023115 0.10697643]
 [0.11356746 0.11953233 0.07680587 0.10427757 0.10041787 0.09535787
  0.09798107 0.09820776 0.08350961 0.11034258]
 [0.0976444  0.11195154 0.09197259 0.10512019 0.09682526 0.0963625
  0.10825687 0.10007548 0.0940638  0.09772733]
 [0.08244316 0.10194953 0.08645782 0.0981131  0.09797924 0.10436712
  0.10916883 0.10044815 0.10250799 0.11656511]
 [0.10495036 0.11110592 0.08655559 0.10945114 0.09636837 0.08939099

INFO:tensorflow:probabilities = [[0.08501897 0.10981774 0.07814597 0.09127724 0.10452519 0.09657007
  0.12744519 0.07985318 0.11184514 0.11550126]
 [0.08979606 0.0997639  0.08912449 0.11163377 0.09840304 0.08731981
  0.11236934 0.10798872 0.08895948 0.11464132]
 [0.09367419 0.10230181 0.09619943 0.08744726 0.10449632 0.10060036
  0.10201313 0.09260354 0.10627407 0.11438983]
 [0.09022234 0.10486937 0.08607627 0.10936718 0.09940822 0.09199367
  0.11101168 0.08900941 0.09352815 0.12451363]
 [0.08845867 0.10195803 0.08903413 0.11497583 0.09592154 0.09280033
  0.1175584  0.0874732  0.10691129 0.10490856]
 [0.09281562 0.1041071  0.09885278 0.10567849 0.09620528 0.08137126
  0.09352579 0.09865324 0.11436733 0.1144231 ]
 [0.08489369 0.09789535 0.09100136 0.11046438 0.10208968 0.09341671
  0.11397801 0.09182597 0.11415984 0.10027496]
 [0.08398905 0.11154797 0.0836056  0.11708106 0.10574009 0.08957407
  0.09668373 0.100618   0.11339429 0.09776613]
 [0.1052134  0.10263537 0.09274545 0.09484039 0.

INFO:tensorflow:global_step/sec: 2.70868
INFO:tensorflow:loss = 2.2672935, step = 201 (36.918 sec)
INFO:tensorflow:probabilities = [[0.09364649 0.09702826 0.10034042 0.10712625 0.11531805 0.07850596
  0.09299996 0.10070251 0.1016098  0.11272241]
 [0.08731023 0.105505   0.08732937 0.09934749 0.11227542 0.09755832
  0.10974003 0.10052254 0.08847631 0.11193532]
 [0.0880444  0.10356509 0.09618477 0.10949083 0.10128221 0.09081311
  0.10414249 0.09252081 0.0989935  0.11496285]
 [0.09443603 0.09932572 0.09393024 0.0996959  0.09005321 0.09621824
  0.10234094 0.1094027  0.11028129 0.10431571]
 [0.08964374 0.10986993 0.08384757 0.11548969 0.09580171 0.09858113
  0.11682378 0.08663201 0.09583566 0.10747474]
 [0.08784509 0.09701635 0.0954213  0.11013289 0.10314099 0.07489956
  0.10370845 0.09461167 0.10197835 0.13124533]
 [0.10091124 0.09847892 0.10966029 0.10478663 0.08049778 0.0945537
  0.0931177  0.11009161 0.09874638 0.10915577]
 [0.09785599 0.0839977  0.08727753 0.10969167 0.10028095 0.093130

INFO:tensorflow:probabilities = [[0.09912101 0.09340706 0.10551694 0.10329629 0.08059575 0.09377331
  0.125336   0.0910736  0.09222437 0.11565567]
 [0.08724251 0.09285199 0.10419347 0.11016574 0.09957245 0.0873966
  0.10636531 0.09641331 0.10558907 0.1102095 ]
 [0.09387168 0.11476128 0.08836451 0.09578151 0.10198866 0.09477455
  0.10947417 0.08983514 0.10908807 0.10206042]
 [0.07827859 0.10164165 0.08778915 0.10901406 0.09937181 0.09789716
  0.11546489 0.09989179 0.10456746 0.10608345]
 [0.13355917 0.1119151  0.08654657 0.10679205 0.08092649 0.09569649
  0.10257865 0.10984612 0.08291311 0.08922615]
 [0.10086342 0.10233939 0.08687678 0.10905806 0.09786493 0.09692979
  0.1063578  0.09390569 0.09937971 0.10642452]
 [0.0877118  0.10872837 0.08940007 0.10306163 0.10785107 0.08702817
  0.10251916 0.09774707 0.10340275 0.11254994]
 [0.08638999 0.10879849 0.09246346 0.11099777 0.10416653 0.09033881
  0.09343769 0.0903936  0.11351474 0.10949898]
 [0.09496512 0.09418948 0.08678438 0.10732386 0.1

<h2>Challenge: Transfer Learning with a Pretrained Model</h2>

Adapted from TensorFlow's <a href="https://www.tensorflow.org/tutorials/image_retraining">retraining tutorial</a>.



Dataset: http://people.eecs.berkeley.edu/~shiry/projects/yearbooks/yearbooks.html. Series of yearbook photos standardized to black and white from 1900s through 2010s. 
        
<h3> Challenge - Classify decade of the photos: </h3>
We have sorted the data into decade here: https://berkeley.box.com/s/boleml1o1ltu5rbfhbt3lbwn51hbce2p

General tips:
1. Set up frame work (from tutorial) to easily try out different NN architectures on the image library
2. The function tf.image.resize_image_with_crop_or_pad() may help resize images to be passable into general NNs (224 x 224 pixels)
3. Look at source code for bigger networks, you can download weights and append layers to the end.

Notes:
- Freezing weights is used to not backpropogate into entire network. This is an important to this tutoriall and will be used in our future research endeavor. This makes it so you do not need to retrain the entire network.

Feel free to comunicate questions you have during the process to us and we want to see your results by 3:00PM next Friday June 22nd!

SLACK: Join channel #nn-bot